## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [29]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Erice,IT,38.0370,12.5865,55.76,94,20,1.14,few clouds
1,1,Cape Town,ZA,-33.9258,18.4232,62.11,88,40,11.50,scattered clouds
2,2,Saint-Philippe,RE,-21.3585,55.7679,71.15,77,15,6.22,few clouds
3,3,Coihaique,CL,-45.5752,-72.0662,55.35,54,0,4.61,clear sky
4,4,Hamilton,US,39.1834,-84.5333,50.99,84,90,1.01,overcast clouds


In [30]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("what is the maximum temperature you prefer for your trip?"))
min_temp = float(input("what is the minimum temperature you prefer for your trip?"))

what is the maximum temperature you prefer for your trip?90
what is the minimum temperature you prefer for your trip?75


In [31]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_city_df = city_data_df.loc[(city_data_df["Max Temp"] <=90) & (city_data_df["Max Temp"] >=75)]
prefered_city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Georgetown,MY,5.4112,100.3354,84.88,84,20,5.75,few clouds
14,14,Biloxi,US,30.3960,-88.8853,75.20,94,90,14.97,mist
23,23,Bathsheba,BB,13.2167,-59.5167,81.21,74,20,13.80,few clouds
28,28,Victoria,HK,22.2855,114.1577,81.03,79,64,1.99,broken clouds
31,31,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,76.41,79,2,8.59,clear sky


In [32]:
# 4a. Determine if there are any empty rows.
prefered_city_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [33]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = prefered_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Georgetown,MY,84.88,few clouds,5.4112,100.3354,
14,Biloxi,US,75.20,mist,30.3960,-88.8853,
23,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
28,Victoria,HK,81.03,broken clouds,22.2855,114.1577,
31,Sao Jose Da Coroa Grande,BR,76.41,clear sky,-8.8978,-35.1478,
36,Ladario,BR,76.28,overcast clouds,-19.0047,-57.6017,
40,Angoche,MZ,77.61,light rain,-16.2325,39.9086,
41,Kovur,IN,78.19,overcast clouds,14.4833,79.9833,
45,Olinda,BR,78.53,broken clouds,-8.0089,-34.8553,
59,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Georgetown,MY,84.88,few clouds,5.4112,100.3354,Cititel Penang
14,Biloxi,US,75.20,mist,30.3960,-88.8853,IP Casino Resort Spa
23,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
28,Victoria,HK,81.03,broken clouds,22.2855,114.1577,Mini Hotel Central
31,Sao Jose Da Coroa Grande,BR,76.41,clear sky,-8.8978,-35.1478,Crôa Mares Hotel


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df =hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Georgetown,MY,84.88,few clouds,5.4112,100.3354,Cititel Penang
14,Biloxi,US,75.20,mist,30.3960,-88.8853,IP Casino Resort Spa
23,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
28,Victoria,HK,81.03,broken clouds,22.2855,114.1577,Mini Hotel Central
31,Sao Jose Da Coroa Grande,BR,76.41,clear sky,-8.8978,-35.1478,Crôa Mares Hotel
...,...,...,...,...,...,...,...
689,Weligama,LK,78.48,broken clouds,5.9667,80.4167,Weligama Bay Resort
693,Ouallam,NE,86.27,scattered clouds,14.3188,2.0866,
700,San Lorenzo,PY,79.74,broken clouds,-25.3333,-57.5333,Hotel del Sur
701,Tucurui,BR,76.19,broken clouds,-3.7661,-49.6725,BBB Rooms Centro Tucurui PA


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Data/vacation_hotel_search.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))